In [ ]:
from sentence_transformers import SentenceTransformer
from matplotlib import pyplot as plt
import pandas as pd
import sys
import os 


In [ ]:
sys.path.insert(0,'/home/salaris/protein_model/')
from functions import url, load_model, predict_sequence

In [ ]:
base_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm2_t6_8M_UR50D_base_model_accelerated/"
# finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/deep-totem-10/"
# finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/volcanic-cherry-11/"
# finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/snowy-snowflake-13/"
finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/sleek-tree-17/"

In [ ]:
model,tokenizer = load_model(base_model,peft_model_path=finetuned_model,only_raw_model=False)

In [ ]:
import torch
def predict_sequence(model, tokenizer, sequence, max_length=512, to_cuda = True):
    """
    Predict the class of the given sequence using the fine-tuned model.
    """
    # Preprocess the input sequence
    inputs = tokenizer(sequence, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    if to_cuda:
        inputs.cuda()
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        
    return logits, predictions.cpu().numpy()

In [ ]:
sequence = ["MKMKIEFSTGCIPISSLFMSIIKEKEKNNKKTKNFTFSVYIKKYSIEGDNFIIEDKVILNISTPDLELGFHIYNGLMTS"]
xx,yy = predict_sequence(model,tokenizer,sequence,max_length=1024,to_cuda=False)

In [ ]:
xx.shape
xx.shape

In [ ]:
output_file = '/home/salaris/protein_model/function_prediction/data/prot_func_validation_with_embedding.h5'

xx = pd.read_hdf(output_file,start = 0 , stop = 5000)

In [ ]:
import numpy as np 
import seaborn as sb
from sklearn.metrics import r2_score
_tmp = xx.sample(100)
sequence = _tmp.sequence.to_list()
em_str = ['e_' + str(n) for n in range(384)]
embeddings =_tmp[em_str].values.tolist()

# preds,_  = predict_sequence(model,tokenizer,sequence,max_length=1024,to_cuda=False)

from matplotlib import pyplot as plt
x = np.array(embeddings).reshape(-1,1)
y =  np.array(preds).reshape(-1,1)
plt.scatter(x,y, s = 1)
# sb.regplot(x = x , y = y )
r2 = r2_score(y_true=  x,y_pred= y)
plt.title(r2)

In [ ]:
x.shape,y.shape,r2,np.sqrt(r2)

In [ ]:
np.array(embeddings).shape

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
xx